In [1]:
from os.path import isfile
import praw

In [11]:
import pandas as pd
r_news = []
pw = open('pw.txt', 'r')
reddit = praw.Reddit(
    client_id="ARq9LWzcl8vmUQ",
    client_secret="tIyiaaYR_tCUQ_5Vy9XI0bM6N1mj_w",
    password=pw,
    user_agent="headliner",
    username="PGCdev",
)
news_subreddit = reddit.subreddit('news')
for post in news_subreddit.top(time_filter='all',limit=30000):
    r_news.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, 
                              post.selftext, post.created])
r_news = pd.DataFrame(r_news,columns=['title', 'score', 'id', 'subreddit', 'url', 
                                                            'num_comments', 'body', 'created'])
r_news['title'].head()

0     Joe Biden elected president of the United States
1    Chauvin found guilty of murder, manslaughter i...
2    President Donald Trump says he has tested posi...
3    Blizzard Employees Staged a Walkout After the ...
4    Trump has left the White House for the last ti...
Name: title, dtype: object

In [12]:
r_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         995 non-null    object 
 1   score         995 non-null    int64  
 2   id            995 non-null    object 
 3   subreddit     995 non-null    object 
 4   url           995 non-null    object 
 5   num_comments  995 non-null    int64  
 6   body          995 non-null    object 
 7   created       995 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 62.3+ KB


In [13]:
import pandas as pd
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import csv #To Convert final table into a csv file to save to your machine
import time
import datetime

In [75]:
#Adapted from this https://gist.github.com/dylankilkenny/3dbf6123527260165f8c5c3bc3ee331b
#This function builds an Pushshift URL, accesses the webpage and stores JSON data in a nested list
def getPushshiftData(after, before, sub):
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/submission/?size=10000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #Print URL to show user
    print(url)
    #Request URL
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    #return the data element which contains all the submissions data
    return data['data']

In [94]:
#This function will be used to extract the key data points from each JSON result
def collectSubData(subm):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    #flairs are not always present so we wrap in try/except
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']

    #Put all data points into a tuple and append to subData
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    #Create a dictionary entry of current submission data and store all data related to it
    subStats[sub_id] = subData

In [95]:
#Create your timestamps and queries for your search URL
#https://www.unixtimestamp.com/index.php > Use this to create your timestamps
after = "1619827200" #Submissions after this timestamp (1577836800 = 01 Jan 20)
before = "1621123200" #Submissions before this timestamp (1607040000 = 04 Dec 20)
sub = "news" #Which Subreddit to search in

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}

In [96]:
# We need to run this function outside the loop first to get the updated after variable
data = getPushshiftData(after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
    after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
    data = getPushshiftData(after, before, sub)
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?size=10000&after=1619827200&before=1621123200&subreddit=news
100
2021-04-30 18:29:46
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1619832586&before=1621123200&subreddit=news
100
2021-04-30 19:42:58
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1619836978&before=1621123200&subreddit=news
100
2021-04-30 21:09:16
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1619842156&before=1621123200&subreddit=news
100
2021-04-30 22:23:20
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1619846600&before=1621123200&subreddit=news
100
2021-04-30 23:37:52
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1619851072&before=1621123200&subreddit=news
100
2021-05-01 00:38:00
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1619854680&before=1621123200&subreddit=news
100
2021-05-01 01:43:17
https://api.pushshift.io/reddit/search/submissio

100
2021-05-03 16:37:57
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620085077&before=1621123200&subreddit=news
100
2021-05-03 17:54:49
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620089689&before=1621123200&subreddit=news
100
2021-05-03 19:06:05
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620093965&before=1621123200&subreddit=news
100
2021-05-03 20:30:28
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620099028&before=1621123200&subreddit=news
100
2021-05-03 21:32:05
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620102725&before=1621123200&subreddit=news
100
2021-05-03 22:33:20
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620106400&before=1621123200&subreddit=news
100
2021-05-03 23:37:54
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620110274&before=1621123200&subreddit=news
100
2021-05-04 00:38:03
https://api.pushshift.io

100
2021-05-06 09:20:36
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620318036&before=1621123200&subreddit=news
100
2021-05-06 10:21:38
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620321698&before=1621123200&subreddit=news
100
2021-05-06 11:29:29
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620325769&before=1621123200&subreddit=news
100
2021-05-06 12:33:22
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620329602&before=1621123200&subreddit=news
100
2021-05-06 13:41:06
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620333666&before=1621123200&subreddit=news
100
2021-05-06 14:53:53
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620338033&before=1621123200&subreddit=news
100
2021-05-06 16:01:11
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620342071&before=1621123200&subreddit=news
100
2021-05-06 17:21:23
https://api.pushshift.io

100
2021-05-09 17:21:11
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620606071&before=1621123200&subreddit=news
100
2021-05-09 18:56:09
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620611769&before=1621123200&subreddit=news
100
2021-05-09 20:42:47
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620618167&before=1621123200&subreddit=news
100
2021-05-09 22:42:27
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620625347&before=1621123200&subreddit=news
100
2021-05-10 00:02:06
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620630126&before=1621123200&subreddit=news
100
2021-05-10 01:27:30
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620635250&before=1621123200&subreddit=news
100
2021-05-10 02:27:17
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620638837&before=1621123200&subreddit=news
100
2021-05-10 03:10:45
https://api.pushshift.io

100
2021-05-13 05:14:20
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620908060&before=1621123200&subreddit=news
100
2021-05-13 06:27:16
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620912436&before=1621123200&subreddit=news
100
2021-05-13 07:31:58
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620916318&before=1621123200&subreddit=news
100
2021-05-13 08:24:31
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620919471&before=1621123200&subreddit=news
100
2021-05-13 09:38:46
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620923926&before=1621123200&subreddit=news
100
2021-05-13 10:38:41
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620927521&before=1621123200&subreddit=news
100
2021-05-13 11:28:27
https://api.pushshift.io/reddit/search/submission/?size=10000&after=1620930507&before=1621123200&subreddit=news
100
2021-05-13 12:40:52
https://api.pushshift.io

In [67]:
subStats

{'l9pnko': [('l9pnko',
   'Priyanka Chopra recalls SP Balasubrahmanyam: ‘Your songs are a huge part of my childhood memories’ - bollywood',
   'https://popularnews.in/priyanka-chopra-recalls-sp-balasubrahmanyam-your-songs-are-a-huge-part-of-my-childhood-memories-bollywood/?feed_id=856196&amp;_unique_id=6017448cedad7',
   'popularnewsindia',
   1,
   datetime.datetime(2021, 1, 31, 16, 0, 13),
   0,
   '/r/news/comments/l9pnko/priyanka_chopra_recalls_sp_balasubrahmanyam_your/',
   'NaN')],
 'l9pnob': [('l9pnob',
   'Priyanka Chopra recalls SP Balasubrahmanyam: ‘Your songs are a huge part of my childhood memories’ - bollywood',
   'https://popularnews.in/priyanka-chopra-recalls-sp-balasubrahmanyam-your-songs-are-a-huge-part-of-my-childhood-memories-bollywood/?feed_id=856196&amp;_unique_id=6017448f21d1f',
   'popularnewsindia',
   1,
   datetime.datetime(2021, 1, 31, 16, 0, 15),
   0,
   '/r/news/comments/l9pnob/priyanka_chopra_recalls_sp_balasubrahmanyam_your/',
   'NaN')],
 'l9ppb2': [('

In [97]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

28637 submissions have added to list
1st entry is:
Plan to End Police Shootings created: 2021-04-30 17:00:04
Last entry is:
Philadelphia says MOVE bombing victims' remains found created: 2021-05-15 16:58:51


In [98]:
def updateSubs_file():
    upload_count = 0
    #location = "\\Reddit Data\\" >> If you're running this outside of a notebook you'll need this to direct to a specific location
    print("input filename of submission file, please add .csv")
    filename = input() #This asks the user what to name the file
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
updateSubs_file()

input filename of submission file, please add .csv
may_1_to_15_news.csv
28637 submissions have been uploaded


In [30]:
pip install psaw

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: certifi>=2017.4.17 in c:\users\webge\anaconda3\lib\site-packages (from requests->psaw) (2020.12.5)



In [31]:
from psaw import PushshiftAPI
api=PushshiftAPI()

In [32]:
gen = api.search_submissions(limit=100)
results = list(gen)

In [38]:
import datetime as dt

start_epoch=int(dt.datetime(2021,1,1).timestamp())

list(api.search_submissions(after=start_epoch, subreddit='news', limit=1))

[submission(all_awardings=[], allow_live_comments=False, author='lnhaxcom', author_flair_css_class=None, author_flair_richtext=[], author_flair_text=None, author_flair_type='text', author_fullname='t2_4gka8z1a', author_patreon_flair=False, author_premium=False, awarders=[], can_mod_post=False, contest_mode=False, created_utc=1621403182, domain='tellygupshup.com', full_link='https://www.reddit.com/r/news/comments/nfxegi/gurfateh_singh_pirzada_wiki_age_girlfriend_wife/', gildings={}, id='nfxegi', is_crosspostable=False, is_meta=False, is_original_content=False, is_reddit_media_domain=False, is_robot_indexable=False, is_self=False, is_video=False, link_flair_background_color='', link_flair_richtext=[], link_flair_text_color='dark', link_flair_type='text', locked=False, media_only=False, no_follow=True, num_comments=0, num_crossposts=0, over_18=False, parent_whitelist_status='all_ads', permalink='/r/news/comments/nfxegi/gurfateh_singh_pirzada_wiki_age_girlfriend_wife/', pinned=False, post_

In [103]:
may = pd.read_csv('may_1_to_15_news.csv')

In [87]:
mar.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
0,lw4hqe,Kkk,https://discord.gg/BWMHA9ua,hellobabykk,1,2021-03-02 07:25:39,0,/r/news/comments/lw4hqe/kkk/,NaN
1,lw4hrs,Rupee Gains 18 Paise To Settle At 73.37 Agains...,https://popularnews.in/rupee-gains-18-paise-to...,popularnewsindia,1,2021-03-02 07:25:42,0,/r/news/comments/lw4hrs/rupee_gains_18_paise_t...,NaN
2,lw4i6l,Novavax Says Covid-19 Vaccine 89% Effective In...,https://popularnews.in/novavax-says-covid-19-v...,popularnewsindia,1,2021-03-02 07:26:09,0,/r/news/comments/lw4i6l/novavax_says_covid19_v...,NaN
3,lw4id6,NY #VİLLOR​​ FÖRSÄLJNİNG İ #ALANYA​​ ( PRİVAT ...,https://tr.pinterest.com/pin/162340761558108404,Ahmetbeyer,1,2021-03-02 07:26:21,1,/r/news/comments/lw4id6/ny_villor_försäljning_...,NaN
4,lw4is4,Snoop Dogg nuthin' but a grandpa thang!!!,https://www.ritrends.com/2021/03/snoop-dogg-nu...,Witty_Ad7272,1,2021-03-02 07:26:47,0,/r/news/comments/lw4is4/snoop_dogg_nuthin_but_...,NaN


In [88]:
mar.loc[mar['Score'].idxmax()]

Post ID                                                             ly0rws
Title                    The United States is now averaging 2 million v...
Url                      https://www.nytimes.com/live/2021/03/04/world/...
Author                                                              rdmc23
Score                                                                78818
Publish Date                                           2021-03-04 17:31:06
Total No. of Comments                                                 4840
Permalink                /r/news/comments/ly0rws/the_united_states_is_n...
Flair                                                         Soft paywall
Name: 4644, dtype: object

In [111]:
mar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Post ID                32573 non-null  object
 1   Title                  32573 non-null  object
 2   Url                    32573 non-null  object
 3   Author                 32573 non-null  object
 4   Score                  32573 non-null  int64 
 5   Publish Date           32573 non-null  object
 6   Total No. of Comments  32573 non-null  int64 
 7   Permalink              32573 non-null  object
 8   Flair                  993 non-null    object
dtypes: int64(2), object(7)
memory usage: 2.2+ MB


In [112]:
jan.append(mar)

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
0,ko10lt,Brexit Becomes Reality As UK Leaves European U...,https://popularnews.in/brexit-becomes-reality-...,popularnewsindia,1,2020-12-31 16:00:01,0,/r/news/comments/ko10lt/brexit_becomes_reality...,NaN
1,ko10oh,COVID-19 UPDATE: Take-out and Delivery OPEN. N...,https://siliconeer.com/current/raj-palace-rest...,siliconeer,1,2020-12-31 16:00:06,0,/r/news/comments/ko10oh/covid19_update_takeout...,NaN
2,ko10sx,Lamborghini Huracan STO Teased Ahead Of Debut,https://popularnews.in/lamborghini-huracan-sto...,popularnewsindia,1,2020-12-31 16:00:15,0,/r/news/comments/ko10sx/lamborghini_huracan_st...,NaN
3,ko11ms,"""It's just utter chaos"": California becomes th...",https://www.cbsnews.com/news/california-become...,Bonboniru,1,2020-12-31 16:01:21,269,/r/news/comments/ko11ms/its_just_utter_chaos_c...,NaN
4,ko11u9,The Number Theory| A decade of rightward shift...,https://popularnews.in/the-number-theory-a-dec...,popularnewsindia,1,2020-12-31 16:01:38,0,/r/news/comments/ko11u9/the_number_theory_a_de...,NaN
...,...,...,...,...,...,...,...,...,...
32568,mhiyms,IDFC First Banks Falls After Fixing Floor Pric...,https://popularnews.in/idfc-first-banks-falls-...,popularnewsindia,1,2021-03-31 16:53:38,0,/r/news/comments/mhiyms/idfc_first_banks_falls...,NaN
32569,mhj0a2,Nigeria to receive 70 million J&amp;J Covid-19...,https://globalfinancialdigest.com/nigeria-to-r...,Globalfinancidigest,1,2021-03-31 16:56:17,0,/r/news/comments/mhj0a2/nigeria_to_receive_70_...,NaN
32570,mhj0bz,Best Join woodworking ideas,http://news.meimei0.info/2021/03/10/best-join-...,Som2ny-Official,1,2021-03-31 16:56:22,0,/r/news/comments/mhj0bz/best_join_woodworking_...,NaN
32571,mhj0jx,"""Uni babes na em even cost pass""",https://newspotng.com/uni-babes-na-em-even-cos...,First-Situation-1384,1,2021-03-31 16:56:43,0,/r/news/comments/mhj0jx/uni_babes_na_em_even_c...,NaN


In [114]:
jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76076 entries, 0 to 76075
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Post ID                76076 non-null  object
 1   Title                  76076 non-null  object
 2   Url                    76076 non-null  object
 3   Author                 76076 non-null  object
 4   Score                  76076 non-null  int64 
 5   Publish Date           76076 non-null  object
 6   Total No. of Comments  76076 non-null  int64 
 7   Permalink              76076 non-null  object
 8   Flair                  2436 non-null   object
dtypes: int64(2), object(7)
memory usage: 5.2+ MB


In [118]:
r_news_data = r_news_data.append(may)
r_news_data

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
0,ko10lt,Brexit Becomes Reality As UK Leaves European U...,https://popularnews.in/brexit-becomes-reality-...,popularnewsindia,1,2020-12-31 16:00:01,0,/r/news/comments/ko10lt/brexit_becomes_reality...,NaN
1,ko10oh,COVID-19 UPDATE: Take-out and Delivery OPEN. N...,https://siliconeer.com/current/raj-palace-rest...,siliconeer,1,2020-12-31 16:00:06,0,/r/news/comments/ko10oh/covid19_update_takeout...,NaN
2,ko10sx,Lamborghini Huracan STO Teased Ahead Of Debut,https://popularnews.in/lamborghini-huracan-sto...,popularnewsindia,1,2020-12-31 16:00:15,0,/r/news/comments/ko10sx/lamborghini_huracan_st...,NaN
3,ko11ms,"""It's just utter chaos"": California becomes th...",https://www.cbsnews.com/news/california-become...,Bonboniru,1,2020-12-31 16:01:21,269,/r/news/comments/ko11ms/its_just_utter_chaos_c...,NaN
4,ko11u9,The Number Theory| A decade of rightward shift...,https://popularnews.in/the-number-theory-a-dec...,popularnewsindia,1,2020-12-31 16:01:38,0,/r/news/comments/ko11u9/the_number_theory_a_de...,NaN
...,...,...,...,...,...,...,...,...,...
28632,ndbmxp,A 10-year-old 《I just want to be a doctor… to ...,https://youtube.com/watch?v=hL77PxtSCzs&amp;fe...,Medium_Vermicelli_36,1,2021-05-15 16:44:53,0,/r/news/comments/ndbmxp/a_10yearold_i_just_wan...,NaN
28633,ndbsdt,Florida man wrestles alligator to save his dog,https://www.nbcnews.com/news/us-news/florida-m...,TVaddictpanda,1,2021-05-15 16:52:45,32,/r/news/comments/ndbsdt/florida_man_wrestles_a...,NaN
28634,ndbuz8,دانلود آهنگ ابر تازه نفزین شد من شدم یه دریا درد,https://pm30music.com/66445/_%d8%a7%d8%a8%d8%b...,pm30music,1,2021-05-15 16:56:45,0,/r/news/comments/ndbuz8/دانلود_آهنگ_ابر_تازه_ن...,NaN
28635,ndbuzp,دانلود آهنگ مازندرانی رضا چنگیزی به نام نازی جان,https://pm30music.com/66505/%d8%b1%d8%b6%d8%a7...,pm30music,1,2021-05-15 16:56:46,0,/r/news/comments/ndbuzp/دانلود_آهنگ_مازندرانی_...,NaN


In [121]:
r_news_data = r_news_data.reset_index()
r_news_data

,level_0,index,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
0,0,0,ko10lt,Brexit Becomes Reality As UK Leaves European U...,https://popularnews.in/brexit-becomes-reality-...,popularnewsindia,1,2020-12-31 16:00:01,0,/r/news/comments/ko10lt/brexit_becomes_reality...,NaN
1,1,1,ko10oh,COVID-19 UPDATE: Take-out and Delivery OPEN. N...,https://siliconeer.com/current/raj-palace-rest...,siliconeer,1,2020-12-31 16:00:06,0,/r/news/comments/ko10oh/covid19_update_takeout...,NaN
2,2,2,ko10sx,Lamborghini Huracan STO Teased Ahead Of Debut,https://popularnews.in/lamborghini-huracan-sto...,popularnewsindia,1,2020-12-31 16:00:15,0,/r/news/comments/ko10sx/lamborghini_huracan_st...,NaN
3,3,3,ko11ms,"""It's just utter chaos"": California becomes th...",https://www.cbsnews.com/news/california-become...,Bonboniru,1,2020-12-31 16:01:21,269,/r/news/comments/ko11ms/its_just_utter_chaos_c...,NaN
4,4,4,ko11u9,The Number Theory| A decade of rightward shift...,https://popularnews.in/the-number-theory-a-dec...,popularnewsindia,1,2020-12-31 16:01:38,0,/r/news/comments/ko11u9/the_number_theory_a_de...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
232801,232801,28632,ndbmxp,A 10-year-old 《I just want to be a doctor… to ...,https://youtube.com/watch?v=hL77PxtSCzs&amp;fe...,Medium_Vermicelli_36,1,2021-05-15 16:44:53,0,/r/news/comments/ndbmxp/a_10yearold_i_just_wan...,NaN
232802,232802,28633,ndbsdt,Florida man wrestles alligator to save his dog,https://www.nbcnews.com/news/us-news/florida-m...,TVaddictpanda,1,2021-05-15 16:52:45,32,/r/news/comments/ndbsdt/florida_man_wrestles_a...,NaN
232803,232803,28634,ndbuz8,دانلود آهنگ ابر تازه نفزین شد من شدم یه دریا درد,https://pm30music.com/66445/_%d8%a7%d8%a8%d8%b...,pm30music,1,2021-05-15 16:56:45,0,/r/news/comments/ndbuz8/دانلود_آهنگ_ابر_تازه_ن...,NaN
232804,232804,28635,ndbuzp,دانلود آهنگ مازندرانی رضا چنگیزی به نام نازی جان,https://pm30music.com/66505/%d8%b1%d8%b6%d8%a7...,pm30music,1,2021-05-15 16:56:46,0,/r/news/comments/ndbuzp/دانلود_آهنگ_مازندرانی_...,NaN


In [123]:
r_news_data.to_csv('r_news_data.csv')